# Features engineer

In [2]:
import pandas as pd
import numpy as np

def quantile_normalize_across(df_list, meta_cols):
    """
    跨多個 DataFrame 做真正的 quantile normalization。
    每個 feature column 分別以所有樣本的 rank 平均值取代。
    """

    # 取出所有 features
    feature_names = [c for c in df_list[0].columns if c not in meta_cols]
    n_features = len(feature_names)

    # 檢查形狀一致
    n_rows = [len(df) for df in df_list]
    if len(set(n_rows)) != 1:
        raise ValueError(f"❌ All DataFrames must have the same number of rows. Got: {n_rows}")

    n = n_rows[0]
    print(f"🧩 Performing quantile normalization on {len(df_list)} datasets, each with {n} rows and {n_features} features")

    # 準備 feature 矩陣列表
    features = [df[feature_names].to_numpy() for df in df_list]

    # 建立一個空間保存平均排序值
    mean_sorted_all = np.zeros((n, n_features))

    # 對每個 feature column 做 rank-based 平均
    for j in range(n_features):
        # 取所有 datasets 的該 feature
        vals = np.vstack([f[:, j] for f in features])
        # 對每個 dataset 內做排序
        sorted_each = np.sort(vals, axis=1)
        # 對 rank 取平均
        mean_sorted_all[:, j] = np.mean(sorted_each, axis=0)

    # 定義函式：把平均 rank 值放回原位置
    def apply_quantile_norm(X, mean_sorted_all):
        X_norm = np.zeros_like(X)
        for j in range(X.shape[1]):
            ranks = np.argsort(np.argsort(X[:, j]))
            X_norm[:, j] = mean_sorted_all[ranks, j]
        return X_norm

    # 套用到所有 DataFrame
    normalized = [apply_quantile_norm(X, mean_sorted_all) for X in features]

    # 回存
    dfs_qn = []
    for df, norm_values in zip(df_list, normalized):
        df_qn = pd.concat([
            df[meta_cols].reset_index(drop=True),
            pd.DataFrame(norm_values, columns=feature_names)
        ], axis=1)
        dfs_qn.append(df_qn)

    return dfs_qn


In [3]:
import os
import pandas as pd

X1 = pd.read_csv("../preprocessed_data/reference/0. raw_data/raw_bigwig/X1_raw_bigwig.tsv", sep="\t")
X2 = pd.read_csv("../preprocessed_data/reference/0. raw_data/raw_bigwig/X2_raw_bigwig.tsv", sep="\t")
X3 = pd.read_csv("../preprocessed_data/reference/0. raw_data/raw_bigwig/X3_raw_bigwig.tsv", sep="\t")

meta_cols = ["gene_name","chr","gene_start","gene_end","TSS_start","TSS_end","strand"]

X1_qn, X2_qn, X3_qn = quantile_normalize_across([X1, X2, X3], meta_cols)

output_path = "../preprocessed_data/reference/1. quantile_normalized/bigwig/"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# 儲存結果
X1_qn.to_csv(f"{output_path}X1_bigwig_qn.tsv", sep="\t", index=False)
X2_qn.to_csv(f"{output_path}X2_bigwig_qn.tsv", sep="\t", index=False)
X3_qn.to_csv(f"{output_path}X3_bigwig_qn.tsv", sep="\t", index=False)


🧩 Performing quantile normalization on 3 datasets, each with 18268 rows and 168 features


In [3]:
import os
import pandas as pd

X1 = pd.read_csv("../preprocessed_data/reference/0. raw_data/raw_bed/X1_with_allMarks_raw.tsv", sep="\t")
X2 = pd.read_csv("../preprocessed_data/reference/0. raw_data/raw_bed/X2_with_allMarks_raw.tsv", sep="\t")
X3 = pd.read_csv("../preprocessed_data/reference/0. raw_data/raw_bed/X3_with_allMarks_raw.tsv", sep="\t")

meta_cols = ["gene_name","chr","gene_start","gene_end","TSS_start","TSS_end","strand"]

X1_qn, X2_qn, X3_qn = quantile_normalize_across([X1, X2, X3], meta_cols)

output_path = "../preprocessed_data/reference/1. quantile_normalized/bed/"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# 儲存結果
X1_qn.to_csv(f"{output_path}X1_bed_qn.tsv", sep="\t", index=False)
X2_qn.to_csv(f"{output_path}X2_bed_qn.tsv", sep="\t", index=False)
X3_qn.to_csv(f"{output_path}X3_bed_qn.tsv", sep="\t", index=False)


🧩 Performing quantile normalization on 3 datasets, each with 18268 rows and 350 features


In [4]:
import os
import pandas as pd

# ============================================================
#                  基本設定
# ============================================================
base_dir = "../preprocessed_data/reference/1. quantile_normalized"
bed_dir = os.path.join(base_dir, "bed")
bw_dir = os.path.join(base_dir, "bigwig")
output_dir = base_dir  # 合併輸出在這層
os.makedirs(output_dir, exist_ok=True)

cells = ["X1", "X2", "X3"]

# 要去除的欄位
drop_cols = ["chr", "gene_start", "gene_end", "TSS_start", "TSS_end", "strand"]

# ============================================================
#                  逐 cell line 合併
# ============================================================
for cell in cells:
    bed_path = os.path.join(bed_dir, f"{cell}_bed_qn.tsv")
    bw_path = os.path.join(bw_dir, f"{cell}_bigwig_qn.tsv")
    out_path = os.path.join(output_dir, f"{cell}_all_qn.tsv")

    print(f"\n=== 🧬 {cell}: merging quantile-normalized bed + bigwig ===")

    if not os.path.exists(bed_path):
        print(f"⚠️ Missing bed file: {bed_path}")
        continue
    if not os.path.exists(bw_path):
        print(f"⚠️ Missing bigwig file: {bw_path}")
        continue

    # --- 讀取 ---
    df_bed = pd.read_csv(bed_path, sep="\t")
    df_bw = pd.read_csv(bw_path, sep="\t")

    if "gene_name" not in df_bed.columns or "gene_name" not in df_bw.columns:
        raise ValueError(f"❌ Missing 'gene_name' column in {cell}")

    print(f"📘 Loaded bed={df_bed.shape}, bigwig={df_bw.shape}")

    # --- 去掉不需要的欄位 ---
    keep_cols = [c for c in df_bed.columns if c not in drop_cols]
    df_bed = df_bed[keep_cols]

    # --- 合併 ---
    df_merged = pd.merge(df_bed, df_bw, on="gene_name", how="outer")
    print(f"✅ Merged shape: {df_merged.shape}")

    # --- 儲存 ---
    df_merged.to_csv(out_path, sep="\t", index=False)
    print(f"💾 Saved → {out_path}")



=== 🧬 X1: merging quantile-normalized bed + bigwig ===
📘 Loaded bed=(18268, 357), bigwig=(18268, 175)
✅ Merged shape: (18268, 525)
💾 Saved → ../preprocessed_data/reference/1. quantile_normalized/X1_all_qn.tsv

=== 🧬 X2: merging quantile-normalized bed + bigwig ===
📘 Loaded bed=(18268, 357), bigwig=(18268, 175)
✅ Merged shape: (18268, 525)
💾 Saved → ../preprocessed_data/reference/1. quantile_normalized/X2_all_qn.tsv

=== 🧬 X3: merging quantile-normalized bed + bigwig ===
📘 Loaded bed=(18268, 357), bigwig=(18268, 175)
✅ Merged shape: (18268, 525)
💾 Saved → ../preprocessed_data/reference/1. quantile_normalized/X3_all_qn.tsv


In [5]:
import os
import pandas as pd

# ============================================================
#                  基本設定
# ============================================================
raw_dir = "../preprocessed_data/reference/0. raw_data"
qn_dir = "../preprocessed_data/reference/1. quantile_normalized"
output_dir = "../preprocessed_data/reference/1. quantile_normalized"
os.makedirs(output_dir, exist_ok=True)

cells = ["X1", "X2"]

# ============================================================
#                  主流程
# ============================================================
for cell in cells:
    y_path = os.path.join(raw_dir, f"{cell}_y.tsv")
    qn_path = os.path.join(qn_dir, f"{cell}_all_qn.tsv")
    out_path = os.path.join(output_dir, f"{cell}_all_qn_with_y.tsv")

    print(f"\n=== 🧬 {cell}: dense normalized rank (0–1) + merge with QN ===")

    if not os.path.exists(y_path):
        print(f"⚠️ Missing Y file: {y_path}")
        continue
    if not os.path.exists(qn_path):
        print(f"⚠️ Missing QN file: {qn_path}")
        continue

    # --- 讀取 ---
    df_y = pd.read_csv(y_path, sep="\t")
    df_qn = pd.read_csv(qn_path, sep="\t")

    if "gene_name" not in df_y.columns or "gene_name" not in df_qn.columns:
        raise ValueError(f"❌ Missing 'gene_name' column in {cell}")
    if "gex" not in df_y.columns:
        raise ValueError(f"❌ Missing 'gex' column in {cell}_y.tsv")

    print(f"📘 Loaded y={df_y.shape}, qn={df_qn.shape}")

    # --- AVERAGE rank: normalize to [0,1], higher gex = higher rank ---
    n_unique = df_y["gex"].nunique()
    df_y["rank_global"] = df_y["gex"].rank(method="average", ascending=True)
    df_y["rank_global"] = (df_y["rank_global"] - 1) / (df_y["rank_global"].max() - 1)

    if "chr" in df_y.columns:
        df_y["rank_chr"] = df_y.groupby("chr")["gex"].rank(method="average", ascending=True)
        df_y["rank_chr"] = df_y.groupby("chr")["rank_chr"].transform(
            lambda x: (x - 1) / (x.max() - 1) if x.max() > 1 else 0
        )
    else:
        print("⚠️ No 'chr' column found in y file — using global rank for rank_chr")
        df_y["rank_chr"] = df_y["rank_global"]

    # --- 合併 ---
    df_merged = pd.merge(df_qn, df_y, on="gene_name", how="left")
    df_merged = df_merged.fillna(0.0)

    # --- 排序輸出 ---
    df_merged = df_merged.sort_values("gene_name").reset_index(drop=True)

    print(f"✅ Merged shape: {df_merged.shape}")
    df_merged.to_csv(out_path, sep="\t", index=False)
    print(f"💾 Saved → {out_path}")



=== 🧬 X1: dense normalized rank (0–1) + merge with QN ===
📘 Loaded y=(16284, 2), qn=(18268, 525)
⚠️ No 'chr' column found in y file — using global rank for rank_chr
✅ Merged shape: (18268, 528)
💾 Saved → ../preprocessed_data/reference/1. quantile_normalized/X1_all_qn_with_y.tsv

=== 🧬 X2: dense normalized rank (0–1) + merge with QN ===
📘 Loaded y=(16284, 2), qn=(18268, 525)
⚠️ No 'chr' column found in y file — using global rank for rank_chr
✅ Merged shape: (18268, 528)
💾 Saved → ../preprocessed_data/reference/1. quantile_normalized/X2_all_qn_with_y.tsv


In [ ]:
df_train_full['rank_global']

0        11560.5
1        11560.5
2        11560.5
3        11560.5
4            0.0
          ...   
18263        0.0
18264        0.0
18265     5742.0
18266    11560.5
18267        0.0
Name: rank_global, Length: 18268, dtype: float64

In [9]:
import os
import json
import random
import pickle   # ✅ 新增
import pandas as pd
import numpy as np
import lightgbm as lgb
from scipy.stats import spearmanr
from datetime import datetime

# ============================================================
#                  GLOBAL REPRODUCIBILITY
# ============================================================
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
print(f"🔒 Global seed set to {SEED}")

# ============================================================
#                  EXPERIMENT CONFIGURATION
# ============================================================
EXPERIMENT_DIR = "../results/lgbm/reference/qn_21"
TRAIN_PATH = "../preprocessed_data/reference/1. quantile_normalized/X2_all_qn_with_y.tsv"
VAL_PATH   = "../preprocessed_data/reference/1. quantile_normalized/X1_all_qn_with_y.tsv"

META_COLS = ["gene_name", "chr", "gene_start", "gene_end",
             "TSS_start", "TSS_end", "strand", "gex", "rank_global", "rank_chr"]
TARGET_COL = "rank_global"

os.makedirs(EXPERIMENT_DIR, exist_ok=True)
LOG_PATH = os.path.join(EXPERIMENT_DIR, "log.txt")

# ✅ 如果已有舊的 log.txt，直接覆蓋（清空）
with open(LOG_PATH, "w") as f:
    f.write("")  # 或寫入 header，如 "==== New Experiment ====\n"

def log(msg):
    """同時印出並寫入 log.txt"""
    print(msg)
    with open(LOG_PATH, "a") as f:
        f.write(f"{msg}\n")


log(f"🚀 Experiment started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
log(f"Experiment directory: {EXPERIMENT_DIR}")

# ============================================================
#                  DATA LOADING
# ============================================================
df_train_full = pd.read_csv(TRAIN_PATH, sep="\t")
df_val_full   = pd.read_csv(VAL_PATH, sep="\t")

# ============================================================
# 🚫 去除 chr1
# ============================================================
before_train = len(df_train_full)
before_val = len(df_val_full)

df_train_full = df_train_full[df_train_full["chr"] != "chr1"].reset_index(drop=True)
df_val_full   = df_val_full[df_val_full["chr"] != "chr1"].reset_index(drop=True)

print(f"🧹 Removed chr1 → train: {before_train} → {len(df_train_full)}, val: {before_val} → {len(df_val_full)}")

feature_cols = [c for c in df_train_full.columns if c not in META_COLS]
log(f"Feature count: {len(feature_cols)}")

# ============================================================
#                  K-FOLD DEFINITION
# ============================================================
chromosomes = [f"chr{i}" for i in range(2, 23)]
folds = [chromosomes[i::5] for i in range(5)]

log("🧩 Chromosome folds:")
for i, fset in enumerate(folds):
    log(f"Fold {i+1}: {fset}")

# ============================================================
#                  LIGHTGBM PARAMETERS
# ============================================================
params = {'verbose': -1,
    'boosting_type': 'gbdt', 'learning_rate': 0.016676974956976915, 'num_leaves': 48, 'max_depth': 8, 'feature_fraction': 0.64561553423692, 'bagging_fraction': 0.8113835038425429, 'bagging_freq': 6, 'lambda_l1': 0.3316673054635859, 'lambda_l2': 0.8969317795206216, 'min_gain_to_split': 0.04923442843722911, 'min_data_in_leaf': 38}


# ============================================================
#                  SAVE CONFIG
# ============================================================
config = {
    "train_path": TRAIN_PATH,
    "val_path": VAL_PATH,
    "seed": SEED,
    "params": params,
    "folds": folds,
    "target_col": TARGET_COL,
}
with open(os.path.join(EXPERIMENT_DIR, "config.json"), "w") as f:
    json.dump(config, f, indent=4)

# ============================================================
#                  TRAINING
# ============================================================
results = []
preds_all = []

for fold_idx, val_chrs in enumerate(folds):
    log(f"\n🚀 Fold {fold_idx+1}/{len(folds)} | Val chromosomes: {val_chrs}")
    fold_dir = os.path.join(EXPERIMENT_DIR, f"fold_{fold_idx+1}")
    os.makedirs(fold_dir, exist_ok=True)

    # Split
    train_chrs = [c for c in chromosomes if c not in val_chrs]
    df_train = df_train_full[df_train_full["chr"].isin(train_chrs)].copy()
    df_val   = df_val_full[df_val_full["chr"].isin(val_chrs)].copy()

    df_val = df_val.reset_index().rename(columns={"index": "orig_idx"})

    X_train, y_train = df_train[feature_cols], df_train[TARGET_COL]
    X_val, y_val     = df_val[feature_cols], df_val[TARGET_COL]

    dtrain = lgb.Dataset(X_train, label=y_train)
    dval   = lgb.Dataset(X_val, label=y_val, reference=dtrain)

    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dtrain, dval],
        valid_names=["train", "val"],
        num_boost_round=2000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=100),
            lgb.log_evaluation(period=200)
        ],
    )

    # Save model (.pkl)
    model_path = os.path.join(fold_dir, "model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    log(f"💾 Model saved: {model_path}")

    # Predict
    df_val["predicted_gex_rank"] = model.predict(X_val, num_iteration=model.best_iteration)
    df_val["fold_id"] = fold_idx + 1

    preds_all.append(df_val[["orig_idx", "gene_name", "predicted_gex_rank", "fold_id"]])

    # ============================================================
    # EVALUATION: Spearman vs rank_global & rank_chr
    # ============================================================
    rho_global = spearmanr(df_val["rank_global"], df_val["predicted_gex_rank"])[0]
    rho_chr_rank = spearmanr(df_val["rank_chr"], df_val["predicted_gex_rank"])[0]

    log(f"📈 Fold {fold_idx+1} Spearman ρ (global) = {rho_global:.4f}")
    log(f"📉 Fold {fold_idx+1} Spearman ρ (chr_rank) = {rho_chr_rank:.4f}")
    log("📊 Per-Chromosome Spearman (vs rank_global):")

    chr_corrs = []
    for chrom, subdf in df_val.groupby("chr"):
        if len(subdf) < 2:
            continue
        rho = spearmanr(subdf["rank_global"], subdf["predicted_gex_rank"])[0]
        chr_corrs.append({"chr": chrom, "spearman": rho})
        log(f"   {chrom:<6s}: ρ = {rho:.4f}")

    results.append({
        "fold": fold_idx + 1,
        "val_chr": val_chrs,
        "spearman_global": rho_global,
        "spearman_chr_rank": rho_chr_rank,
        "per_chr": chr_corrs
    })

# ============================================================
#                  SAVE PREDICTIONS & SUMMARY
# ============================================================

# ============================================================
# 🔁 合併並依照原始順序還原
# ============================================================
df_preds_all = pd.concat(preds_all, ignore_index=True)
df_preds_all = df_preds_all.sort_values("orig_idx").reset_index(drop=True)

# 驗證對齊
assert len(df_preds_all) == len(df_val_full)
assert (df_preds_all["gene_name"].values == df_val_full["gene_name"].values).all(), \
    "❌ Prediction order mismatch with df_val_full!"

# ✅ 輸出只有 gene_name + predicted_gex_rank
df_preds_all[["gene_name", "predicted_gex_rank"]].to_csv(
    os.path.join(EXPERIMENT_DIR, "predict_val.tsv"), sep="\t", index=False
)
log("💾 Final predictions (aligned) saved (predict_val.tsv)")

# ✅ 全域 Spearman：用原始 df_val_full 的真值 vs 你的預測
overall_val_spearman = spearmanr(
    df_val_full["rank_global"], 
    df_preds_all["predicted_gex_rank"]
)[0]
log(f"🌍 Overall validation Spearman (aligned) = {overall_val_spearman:.4f}")



summary_fold = pd.DataFrame([
    {"fold": r["fold"], "val_chr": ",".join(r["val_chr"]), "overall_spearman": r["overall_spearman"]}
    for r in results
])

chr_rows = []
for r in results:
    for c in r["per_chr"]:
        chr_rows.append({"fold": r["fold"], "chr": c["chr"], "spearman": c["spearman"]})
summary_chr = pd.DataFrame(chr_rows)

chr_mean = summary_chr.groupby("chr", as_index=False)["spearman"].mean()
chr_mean.rename(columns={"spearman": "mean_spearman"}, inplace=True)

# summary_output = summary_fold.merge(chr_mean, how="cross")
# summary_output.to_csv(os.path.join(EXPERIMENT_DIR, "summary.tsv"), sep="\t", index=False)

# log("💾 Summary saved (summary.tsv)")
log(f"Mean Spearman (across folds): {summary_fold['overall_spearman'].mean():.4f}")
log(f"Overall mean of per-chromosome means: {chr_mean['mean_spearman'].mean():.4f}")

log(f"\n✅ Experiment finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


🔒 Global seed set to 42
🚀 Experiment started at 2025-10-14 23:53:51
Experiment directory: ../results/lgbm/reference/qn_21
🧹 Removed chr1 → train: 18268 → 16284, val: 18268 → 16284
Feature count: 518
🧩 Chromosome folds:
Fold 1: ['chr2', 'chr7', 'chr12', 'chr17', 'chr22']
Fold 2: ['chr3', 'chr8', 'chr13', 'chr18']
Fold 3: ['chr4', 'chr9', 'chr14', 'chr19']
Fold 4: ['chr5', 'chr10', 'chr15', 'chr20']
Fold 5: ['chr6', 'chr11', 'chr16', 'chr21']

🚀 Fold 1/5 | Val chromosomes: ['chr2', 'chr7', 'chr12', 'chr17', 'chr22']
Training until validation scores don't improve for 100 rounds
[200]	train's l2: 0.0159191	val's l2: 0.0292287
Early stopping, best iteration is:
[186]	train's l2: 0.0163488	val's l2: 0.0291883
💾 Model saved: ../results/lgbm/reference/qn_21/fold_1/model.pkl
📈 Fold 1 Spearman ρ (global) = 0.7229
📉 Fold 1 Spearman ρ (chr_rank) = 0.7229
📊 Per-Chromosome Spearman (vs rank_global):
   chr12 : ρ = 0.7380
   chr17 : ρ = 0.7016
   chr2  : ρ = 0.7177
   chr22 : ρ = 0.7345
   chr7  : ρ 

KeyError: 'overall_spearman'

In [24]:
import optuna
import lightgbm as lgb
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import warnings, json, os, random
warnings.filterwarnings("ignore")

# ============================================================
#                  CONFIGURATION
# ============================================================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

TRAIN_PATH = "../preprocessed_data/CAGE-merged/4. features selection/X1_all_rank_features_pruned.tsv"
VAL_PATH   = "../preprocessed_data/CAGE-merged/4. features selection/X2_all_rank_features_pruned.tsv"

META_COLS = ["gene_name", "chr", "gene_start", "gene_end",
             "TSS_start", "TSS_end", "strand", "gex", "gex_rank"]
TARGET_COL = "gex_rank"

# ============================================================
#                  LOAD DATA
# ============================================================
df_train = pd.read_csv(TRAIN_PATH, sep="\t")
df_val   = pd.read_csv(VAL_PATH, sep="\t")

# 合併訓練 + 驗證作為整體 pool（我們會從中抽 chr）
df_all = pd.concat([df_train, df_val], ignore_index=True)
chromosomes = sorted(df_all["chr"].unique())

print(f"🧬 Total chromosomes in dataset: {chromosomes}")
print(f"Total samples: {len(df_all):,}")

feature_cols = [c for c in df_all.columns if c not in META_COLS]

# ============================================================
#                  OBJECTIVE FUNCTION
# ============================================================
def objective(trial):
    """Optuna optimization objective for LightGBM hyperparameters"""

    # 1️⃣ 隨機抽取 4 個 chromosome 當 validation
    # 為每個 trial 建立獨立的隨機種子（可重現）
    rng = random.Random(SEED)
    val_chrs = rng.sample(chromosomes, 4)
    train_chrs = [c for c in chromosomes if c not in val_chrs]
    df_train_fold = df_all[df_all["chr"].isin(train_chrs)].copy()
    df_val_fold   = df_all[df_all["chr"].isin(val_chrs)].copy()

    X_train, y_train = df_train_fold[feature_cols], df_train_fold[TARGET_COL]
    X_val, y_val     = df_val_fold[feature_cols], df_val_fold[TARGET_COL]

    # 2️⃣ 定義超參數搜尋空間
    params = {
        "objective": "regression",
        "metric": "rmse",
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 16, 128, step=8),
        "max_depth": trial.suggest_int("max_depth", -1, 12),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 1.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 2.0),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.0, 0.05),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "verbosity": -1,
        "seed": SEED,
        "deterministic": True,
        "force_row_wise": True,
    }

    # 3️⃣ 訓練模型
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)

    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dval],
        num_boost_round=3000,
        callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)],
    )

    # 4️⃣ 評估 Spearman correlation
    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
    rho_overall = spearmanr(y_val, y_pred)[0]

    # 🔍 也可計算每個 chr 的平均 Spearman（可開啟）
    per_chr_rho = []
    for chrom, subdf in df_val_fold.assign(pred=y_pred).groupby("chr"):
        if len(subdf) > 2:
            per_chr_rho.append(spearmanr(subdf["gex_rank"], subdf["pred"])[0])
    mean_rho = np.nanmean(per_chr_rho)

    # 你可以切換回傳哪個：
    return mean_rho if not np.isnan(mean_rho) else rho_overall


# ============================================================
#                  RUN OPTIMIZATION
# ============================================================
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=300, show_progress_bar=True)

# ============================================================
#                  SHOW BEST RESULT
# ============================================================
print("✅ Best Spearman:", study.best_value)
print("🏆 Best Parameters:")
for k, v in study.best_params.items():
    print(f"   {k}: {v}")

# ============================================================
#                  SAVE RESULT
# ============================================================
SAVE_DIR = "../results/lgbm/optuna_qn_random4chr"
os.makedirs(SAVE_DIR, exist_ok=True)

study.trials_dataframe().to_csv(os.path.join(SAVE_DIR, "trials_log.csv"), index=False)
with open(os.path.join(SAVE_DIR, "best_params.json"), "w") as f:
    json.dump(study.best_params, f, indent=4)

print(f"💾 Saved best params and log to {SAVE_DIR}")


[I 2025-10-14 00:35:10,754] A new study created in memory with name: no-name-1a41df6d-1e5d-461d-8503-301a45b7d576


🧬 Total chromosomes in dataset: ['chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9']
Total samples: 32,568


Best trial: 0. Best value: 0.715367:   0%|          | 1/300 [00:20<1:42:05, 20.49s/it]

[I 2025-10-14 00:35:31,242] Trial 0 finished with value: 0.7153666975380131 and parameters: {'boosting_type': 'dart', 'learning_rate': 0.026975154833351143, 'num_leaves': 80, 'max_depth': 1, 'feature_fraction': 0.662397808134481, 'bagging_fraction': 0.5290418060840998, 'bagging_freq': 7, 'lambda_l1': 0.6011150117432088, 'lambda_l2': 1.416145155592091, 'min_gain_to_split': 0.0010292247147901223, 'min_data_in_leaf': 98}. Best is trial 0 with value: 0.7153666975380131.


Best trial: 1. Best value: 0.754737:   1%|          | 2/300 [00:34<1:22:38, 16.64s/it]

...

[I 2025-10-14 03:10:52,253] Trial 297 finished with value: 0.7616697949929448 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.00843325066877965, 'num_leaves': 32, 'max_depth': 12, 'feature_fraction': 0.9312379260810413, 'bagging_fraction': 0.8903566555267518, 'bagging_freq': 4, 'lambda_l1': 0.32390668551719937, 'lambda_l2': 0.47716967310558817, 'min_gain_to_split': 0.03532470477143433, 'min_data_in_leaf': 53}. Best is trial 156 with value: 0.7636242801608152.


Best trial: 156. Best value: 0.763624: 100%|█████████▉| 299/300 [2:36:19<00:37, 37.57s/it]

[I 2025-10-14 03:11:30,274] Trial 298 finished with value: 0.762417035193625 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.007310058336849808, 'num_leaves': 40, 'max_depth': 12, 'feature_fraction': 0.9242031727726934, 'bagging_fraction': 0.9143208269055778, 'bagging_freq': 3, 'lambda_l1': 0.30197087430468506, 'lambda_l2': 0.4325693848531994, 'min_gain_to_split': 0.033948898140069995, 'min_data_in_leaf': 50}. Best is trial 156 with value: 0.7636242801608152.


Best trial: 156. Best value: 0.763624: 100%|██████████| 300/300 [2:37:02<00:00, 31.41s/it]

[I 2025-10-14 03:12:13,419] Trial 299 finished with value: 0.7610446943185261 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.007867451831643453, 'num_leaves': 40, 'max_depth': 12, 'feature_fraction': 0.9179670432850212, 'bagging_fraction': 0.9171654478438177, 'bagging_freq': 3, 'lambda_l1': 0.285062402366424, 'lambda_l2': 0.21583287814255672, 'min_gain_to_split': 0.03336515504924283, 'min_data_in_leaf': 48}. Best is trial 156 with value: 0.7636242801608152.
✅ Best Spearman: 0.7636242801608152
🏆 Best Parameters:
   boosting_type: gbdt
   learning_rate: 0.015470327280019703
   num_leaves: 40
   max_depth: 9
   feature_fraction: 0.6807426852258752
   bagging_fraction: 0.9055094284104167
   bagging_freq: 6
   lambda_l1: 0.3182565921517314
   lambda_l2: 0.06834504087496518
   min_gain_to_split: 0.02379360490236626
   min_data_in_leaf: 64
💾 Saved best params and log to ../results/lgbm/optuna_qn_random4chr


In [ ]:
import os
import json
import random
import pickle
import pandas as pd
import numpy as np
import lightgbm as lgb
from scipy.stats import spearmanr
from datetime import datetime

# ============================================================
#                  GLOBAL CONFIG
# ============================================================
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
print(f"🔒 Global seed set to {SEED}")

# ============================================================
#                  PATHS
# ============================================================
EXPERIMENT_DIR = "../results/lgbm/leave_one_chr"
TRAIN_PATH = "../preprocessed_data/CAGE-merged/4. features selection/X1_all_rank_features_pruned.tsv"
VAL_PATH   = "../preprocessed_data/CAGE-merged/4. features selection/X2_all_rank_features_pruned.tsv"

META_COLS = ["gene_name", "chr", "gene_start", "gene_end",
             "TSS_start", "TSS_end", "strand", "gex", "gex_rank"]
TARGET_COL = "gex_rank"

os.makedirs(EXPERIMENT_DIR, exist_ok=True)
LOG_PATH = os.path.join(EXPERIMENT_DIR, "log.txt")
with open(LOG_PATH, "w") as f:
    f.write("==== Leave-One-Chromosome-Out Experiment ====\n")

def log(msg):
    print(msg)
    with open(LOG_PATH, "a") as f:
        f.write(f"{msg}\n")

# ============================================================
#                  DATA LOADING
# ============================================================
df_train_full = pd.read_csv(TRAIN_PATH, sep="\t")
df_val_full   = pd.read_csv(VAL_PATH, sep="\t")

feature_cols = [c for c in df_train_full.columns if c not in META_COLS]
chromosomes = [f"chr{i}" for i in range(2, 23)]
log(f"📚 Loaded data with {len(chromosomes)} chromosomes.")
log(f"Feature count: {len(feature_cols)}")

# ============================================================
#                  LIGHTGBM PARAMS
# ============================================================
params = {
    "objective": "regression",
    "metric": "rmse",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 16,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 3,
    "seed": SEED,
    "deterministic": True,
    "force_row_wise": True,
}

# ============================================================
#                  EXPERIMENT LOOP
# ============================================================
overall_results = []
preds_all = []

for test_chr in chromosomes:
    log(f"\n🧪 Outer Loop → Test chromosome: {test_chr}")
    outer_dir = os.path.join(EXPERIMENT_DIR, f"test_{test_chr}")
    os.makedirs(outer_dir, exist_ok=True)

    # 剩下的 20 個 chromosome 用於 inner CV
    inner_chrs = [c for c in chromosomes if c != test_chr]
    folds = [inner_chrs[i::5] for i in range(5)]
    log(f"Inner folds (5-fold over 20 chr): {folds}")

    test_df = df_val_full[df_val_full["chr"] == test_chr].copy()
    test_X = test_df[feature_cols]
    fold_preds = []

    for fold_idx, val_chrs in enumerate(folds):
        log(f"\n🚀 Fold {fold_idx+1}/5 | Val chromosomes: {val_chrs}")
        fold_dir = os.path.join(outer_dir, f"fold_{fold_idx+1}")
        os.makedirs(fold_dir, exist_ok=True)

        train_chrs = [c for c in inner_chrs if c not in val_chrs]
        df_train = df_train_full[df_train_full["chr"].isin(train_chrs)]
        df_val = df_train_full[df_train_full["chr"].isin(val_chrs)]

        X_train, y_train = df_train[feature_cols], df_train[TARGET_COL]
        X_val, y_val = df_val[feature_cols], df_val[TARGET_COL]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)

        model = lgb.train(
            params,
            dtrain,
            valid_sets=[dtrain, dval],
            valid_names=["train", "val"],
            num_boost_round=2000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=100),
                lgb.log_evaluation(period=200)
            ],
        )

        model_path = os.path.join(fold_dir, "model.pkl")
        with open(model_path, "wb") as f:
            pickle.dump(model, f)
        log(f"💾 Model saved: {model_path}")

        val_pred = model.predict(X_val, num_iteration=model.best_iteration)
        val_spearman = spearmanr(y_val, val_pred)[0]
        log(f"📈 Fold {fold_idx+1} inner val Spearman = {val_spearman:.4f}")

        # 對 test_chr 預測
        test_pred = model.predict(test_X, num_iteration=model.best_iteration)
        fold_preds.append(test_pred)

    # =======================================================
    # 平均五個模型對 test_chr 的預測
    # =======================================================
    mean_pred = np.mean(np.vstack(fold_preds), axis=0)
    test_df["predicted_gex_rank"] = mean_pred
    test_df["test_chr"] = test_chr
    preds_all.append(test_df[["gene_name", "chr", "predicted_gex_rank"]])

    # 計算 Spearman
    test_spearman = spearmanr(test_df["gex_rank"], test_df["predicted_gex_rank"])[0]
    overall_results.append({"test_chr": test_chr, "spearman": test_spearman})
    log(f"🌍 Test chromosome {test_chr}: Spearman ρ = {test_spearman:.4f}")

# ============================================================
#                  SUMMARY
# ============================================================
df_preds_all = pd.concat(preds_all, ignore_index=True)
df_preds_all.to_csv(os.path.join(EXPERIMENT_DIR, "predict_all.tsv"), sep="\t", index=False)

df_summary = pd.DataFrame(overall_results)
df_summary.to_csv(os.path.join(EXPERIMENT_DIR, "summary.tsv"), sep="\t", index=False)

mean_rho = df_summary["spearman"].mean()
log(f"\n✅ Overall mean Spearman across 21 test chromosomes: {mean_rho:.4f}")
log(f"Experiment completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


🔒 Global seed set to 42
📚 Loaded data with 21 chromosomes.
Feature count: 277

🧪 Outer Loop → Test chromosome: chr2
Inner folds (5-fold over 20 chr): [['chr3', 'chr8', 'chr13', 'chr18'], ['chr4', 'chr9', 'chr14', 'chr19'], ['chr5', 'chr10', 'chr15', 'chr20'], ['chr6', 'chr11', 'chr16', 'chr21'], ['chr7', 'chr12', 'chr17', 'chr22']]

🚀 Fold 1/5 | Val chromosomes: ['chr3', 'chr8', 'chr13', 'chr18']
Training until validation scores don't improve for 100 rounds
[200]	train's rmse: 0.123313	val's rmse: 0.145375
Early stopping, best iteration is:
[156]	train's rmse: 0.127639	val's rmse: 0.145251
💾 Model saved: ../results/lgbm/leave_one_chr/test_chr2/fold_1/model.pkl
📈 Fold 1 inner val Spearman = 0.7779

🚀 Fold 2/5 | Val chromosomes: ['chr4', 'chr9', 'chr14', 'chr19']
Training until validation scores don't improve for 100 rounds
[200]	train's rmse: 0.121314	val's rmse: 0.14863
[400]	train's rmse: 0.103246	val's rmse: 0.148088
Early stopping, best iteration is:
[428]	train's rmse: 0.101137	val